<h1><b><center>LUNG CANCER PREDICTION<center><b></h1>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,precision_score,recall_score,f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score,GridSearchCV

In [ ]:
lung_data = pd.read_csv('/content/lung_data_cleaned.csv')

In [ ]:
lung_data.head(5)

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,1,69,0,1,1,0,0,1,0,1,1,1,1,1,1,1
1,1,74,1,0,0,0,1,1,1,0,0,0,1,1,1,1
2,0,59,0,0,0,1,0,1,0,1,0,1,1,0,1,0
3,1,63,1,1,1,0,0,0,0,0,1,0,0,1,1,0
4,0,63,0,1,0,0,0,0,0,1,0,1,1,0,0,0


#Data Cleaning

2 ------------------> Yes
1 ----------------------> No

In [ ]:
df = pd.read_csv('/content/survey lung cancer.csv')

In [ ]:
print(df.columns)

Index(['GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY',
       'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
       'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
       'SWALLOWING DIFFICULTY', 'CHEST PAIN', 'LUNG_CANCER'],
      dtype='object')


In [ ]:
df['GENDER'] = df['GENDER'].map({'M':1,'F':0})
df['SMOKING'] = df['SMOKING'].map({2: 1, 1: 0})
df['YELLOW_FINGERS'] = df['YELLOW_FINGERS'].map({2: 1, 1: 0})
df['ANXIETY'] = df['ANXIETY'].map({2: 1, 1: 0})
df['PEER_PRESSURE'] = df['PEER_PRESSURE'].map({2: 1, 1: 0})
df['CHRONIC DISEASE'] = df['CHRONIC DISEASE'].map({2: 1, 1: 0})
df['FATIGUE '] = df['FATIGUE '].map({2: 1, 1: 0})
df['ALLERGY '] = df['ALLERGY '].map({2: 1, 1: 0})
df['WHEEZING'] = df['WHEEZING'].map({2: 1, 1: 0})
df['ALCOHOL CONSUMING'] = df['ALCOHOL CONSUMING'].map({2: 1, 1: 0})
df['COUGHING'] = df['COUGHING'].map({2: 1, 1: 0})
df['SHORTNESS OF BREATH'] = df['SHORTNESS OF BREATH'].map({2: 1, 1: 0})
df['SWALLOWING DIFFICULTY'] = df['SWALLOWING DIFFICULTY'].map({2: 1, 1: 0})
df['CHEST PAIN'] = df['CHEST PAIN'].map({2: 1, 1: 0})
df['LUNG_CANCER'] = df['LUNG_CANCER'].map({'YES': 1, 'NO': 0})

In [ ]:
print(df.head())

   GENDER  AGE  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0       1   69        0               1        1              0   
1       1   74        1               0        0              0   
2       0   59        0               0        0              1   
3       1   63        1               1        1              0   
4       0   63        0               1        0              0   

   CHRONIC DISEASE  FATIGUE   ALLERGY   WHEEZING  ALCOHOL CONSUMING  COUGHING  \
0                0         1         0         1                  1         1   
1                1         1         1         0                  0         0   
2                0         1         0         1                  0         1   
3                0         0         0         0                  1         0   
4                0         0         0         1                  0         1   

   SHORTNESS OF BREATH  SWALLOWING DIFFICULTY  CHEST PAIN  LUNG_CANCER  
0                    1               

In [ ]:
# Save the updated DataFrame to a new CSV file
df.to_csv('lung_data_cleaned.csv', index=False)

In [ ]:
lung_data.shape

(309, 16)

In [ ]:
lung_data.describe()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
count,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000
mean,0.524272,62.673139,0.563107,0.569579,0.498382,0.501618,0.504854,0.673139,0.556634,0.556634,0.556634,0.579288,0.640777,0.469256,0.556634,0.873786
std,0.500221,8.210301,0.496806,0.495938,0.500808,0.500808,0.500787,0.469827,0.497588,0.497588,0.497588,0.494474,0.480551,0.499863,0.497588,0.332629
min,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,62.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
75%,1.000000,69.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,87.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
lung_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309 entries, 0 to 308
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   GENDER                 309 non-null    int64
 1   AGE                    309 non-null    int64
 2   SMOKING                309 non-null    int64
 3   YELLOW_FINGERS         309 non-null    int64
 4   ANXIETY                309 non-null    int64
 5   PEER_PRESSURE          309 non-null    int64
 6   CHRONIC DISEASE        309 non-null    int64
 7   FATIGUE                309 non-null    int64
 8   ALLERGY                309 non-null    int64
 9   WHEEZING               309 non-null    int64
 10  ALCOHOL CONSUMING      309 non-null    int64
 11  COUGHING               309 non-null    int64
 12  SHORTNESS OF BREATH    309 non-null    int64
 13  SWALLOWING DIFFICULTY  309 non-null    int64
 14  CHEST PAIN             309 non-null    int64
 15  LUNG_CANCER            309 non-null    i

In [ ]:
lung_data.isnull().sum()

,0
GENDER,0
AGE,0
SMOKING,0
YELLOW_FINGERS,0
ANXIETY,0
PEER_PRESSURE,0
CHRONIC DISEASE,0
FATIGUE,0
ALLERGY,0
WHEEZING,0


In [ ]:
# Check for NaN values
print(lung_data.isna().sum())

# Check for infinite values
print(np.isinf(lung_data).sum())

GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL CONSUMING        0
COUGHING                 0
SHORTNESS OF BREATH      0
SWALLOWING DIFFICULTY    0
CHEST PAIN               0
LUNG_CANCER              0
dtype: int64
GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL CONSUMING        0
COUGHING                 0
SHORTNESS OF BREATH      0
SWALLOWING DIFFICULTY    0
CHEST PAIN               0
LUNG_CANCER              0
dtype: int64


*Removing the Dupicate*

In [ ]:
duplicate_count = lung_data.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicate_count}")


Number of duplicate rows: 33


In [ ]:
# Correct way to remove duplicates in place
lung_data.drop_duplicates(inplace=True)
print(lung_data.shape)

(276, 16)


In [ ]:
lung_data['LUNG_CANCER'].value_counts()

,count
LUNG_CANCER,
1,238
0,38


In [ ]:
X = lung_data.drop(columns = ['LUNG_CANCER'],axis=1)
Y = lung_data['LUNG_CANCER']

*Spliting DataSet for Training and Testing*

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2, random_state=2)

*Standardizing the Data*

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(276, 15) (220, 15) (56, 15)


In [ ]:
print(X_train)

[[-1.06579657  0.00692638 -1.08545743 ...  0.77835687 -0.92127059
  -1.13651514]
 [-1.06579657 -1.63409199 -1.08545743 ...  0.77835687 -0.92127059
  -1.13651514]
 [ 0.93826536  0.59300436 -1.08545743 ... -1.28475772 -0.92127059
  -1.13651514]
 ...
 [ 0.93826536  0.47578877 -1.08545743 ...  0.77835687 -0.92127059
   0.87988269]
 [-1.06579657 -1.75130759 -1.08545743 ...  0.77835687  1.08545743
  -1.13651514]
 [-1.06579657 -0.69636721 -1.08545743 ... -1.28475772 -0.92127059
  -1.13651514]]


# **Model Selection**

*Logistic Regression*

In [ ]:
# model = LogisticRegression() # Model Giving Low accuracy   50 range

*Support Vector Classifier*

In [ ]:
# model  = SVC(kernel = 'linear')# Model Giving Low accuracy   50 range

*GridSearchCV*

In [ ]:
# from sklearn.model_selection import GridSearchCV
# model = SVC()
# parameters = {
#     'kernel':['linear','sigmoid','poly','rbf'],
#     'C':[1,5,10,20]
# }
# #grid search
# grid = GridSearchCV(model,parameters,cv=5)
# grid.fit(X_train,y_train)
# # loading the data into pandas data frame
# data = pd.DataFrame(grid.cv_results_)
# data.head()
# # grid search result
# data[['param_C','param_kernel','mean_test_score']]

*Random Forest Classifier*

In [ ]:
# model = RandomForestClassifier(n_estimators=100,max_depth=5,min_samples_split=10,min_samples_leaf=4)  # Model uccur overfitting issues

*CatBoost (Categorical Boosting)*

In [ ]:
# # If you know which columns are categorical (e.g., 'column_1', 'column_2'):
# categorical_features = ['SMOKING'	,'YELLOW_FINGERS',	'ANXIETY',	'PEER_PRESSURE',	'CHRONIC_DISEASE',	'FATIGUE',	'ALLERGY',	'WHEEZING',	'ALCOHOL_CONSUMING',	'COUGHING',	'SHORTNESS_OF_BREATH',	'SWALLOWING_DIFFICULTY'	,'CHEST_PAIN']  # Column names of categorical features

# # Alternatively, you can use the column index positions:
# # categorical_features = [0, 2]  # Index positions of categorical features
# # Initialize CatBoostClassifier with basic parameters
# model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, eval_metric='Accuracy')

# # Train the model
# model.fit(X_train, y_train, cat_features=categorical_features, eval_set=(X_test, y_test), verbose=50)

*XGBoost (eXtreme Gradient Boosting)*

In [ ]:
# model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
# model.fit(X_train, y_train)
# model = xgb.XGBClassifier(reg_lambda=1, reg_alpha=0.5)  # L2 and L1 regularization

*Decision Trees*

In [ ]:
# model = DecisionTreeClassifier(max_depth = 10, min_samples_leaf = 4)

# Model Training

**eXtreme Gradient Boosting**

In [ ]:
model = xgb.XGBClassifier(reg_lambda=1, reg_alpha=0.5)  # L2 and L1 regularization

In [ ]:
model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

**Cross Validation**

In [ ]:
scores = cross_val_score(model, X, Y, cv=5)  # 5-fold cross-validation
print(f"Cross-validation accuracy: {scores}")

Cross-validation accuracy: [0.875      0.85454545 0.89090909 0.89090909 0.90909091]


# *Feature Selection*

In [ ]:
# print(lung_data.columns)
# feature_importances = model.feature_importances_
# sorted_indices = np.argsort(feature_importances)[::-1]
# top_features = sorted_indices[:12]  # Use the top 10 most important features
# print(X.columns[top_features])

In [ ]:
# from sklearn.model_selection import GridSearchCV
# param_grid = {
#     'n_estimators': [100, 200],
#     'max_depth': [10, 20],
#     'min_samples_split': [2, 5],
#     'min_samples_leaf': [1, 2]
# }
# grid_search = GridSearchCV(model, param_grid, cv=3)
# grid_search.fit(X_train, y_train)
# print(grid_search.best_params_)

In [ ]:
model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

# **Model Evaluation**

*Training Data*

In [ ]:
print("Model Evaluation on Training Data")
print("----------------------------------")
y_train_pred = model.predict(X_train)
print("Confusion Matrix:\n",confusion_matrix(y_train,y_train_pred))
report = classification_report(y_train, y_train_pred)
print("Classification Report:\n", report)
print("Accuracy Score on XGBoost (eXtreme Gradient Boosting): ",accuracy_score(y_train,y_train_pred))
print("Precision: ",precision_score(y_train,y_train_pred))
print("Recall: ",recall_score(y_train,y_train_pred))
print("F1 score: ",f1_score(y_train,y_train_pred))
print("roc_auc_score",roc_auc_score(y_train,y_train_pred))

Model Evaluation on Training Data
----------------------------------
Confusion Matrix:
 [[ 30   2]
 [  1 187]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.94      0.95        32
           1       0.99      0.99      0.99       188

    accuracy                           0.99       220
   macro avg       0.98      0.97      0.97       220
weighted avg       0.99      0.99      0.99       220

Accuracy Score on XGBoost (eXtreme Gradient Boosting):  0.9863636363636363
Precision:  0.9894179894179894
Recall:  0.9946808510638298
F1 score:  0.9920424403183024
roc_auc_score 0.9660904255319149


*Testing Data*

In [ ]:
print("Model Evaluation on Testing Data")
print("----------------------------------")
y_test_pred = model.predict(X_test)
print("Confusion Matrix:\n",confusion_matrix(y_test,y_test_pred))
report = classification_report(y_test, y_test_pred)
print("Classification Report:\n", report)
print("Accuracy Score on XGBoost (eXtreme Gradient Boosting): ",accuracy_score(y_test,y_test_pred))
print("Precision: ",precision_score(y_test,y_test_pred))
print("Recall: ",recall_score(y_test,y_test_pred))
print("F1 score: ",f1_score(y_test,y_test_pred))
print("roc_auc_score",roc_auc_score(y_test,y_test_pred))

Model Evaluation on Testing Data
----------------------------------
Confusion Matrix:
 [[ 3  3]
 [ 1 49]]
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.50      0.60         6
           1       0.94      0.98      0.96        50

    accuracy                           0.93        56
   macro avg       0.85      0.74      0.78        56
weighted avg       0.92      0.93      0.92        56

Accuracy Score on XGBoost (eXtreme Gradient Boosting):  0.9285714285714286
Precision:  0.9423076923076923
Recall:  0.98
F1 score:  0.9607843137254902
roc_auc_score 0.74
